In [37]:
import numpy as np
import spacy
from datasets import load_dataset

PROB = 0
WORD = 1
UNIGRAM = 1
BIGRAM = 2

In [38]:
class Corpus:
    def __init__(self):
        self.nlp = None
        self.unigram_data = {}  # {w:count}
        self.bigram_data = {}  # {w_prev: {w:count}}
        self.M = 0
        self.m = 0
        self.unigram_data_size = 0
        self.bigram_data_size = {}

    def add_to_unigram(self, w):
        self.unigram_data_size += 1
        if w in self.unigram_data:
            self.unigram_data[w] += 1
        else:
            self.unigram_data[w] = 1

    def add_to_bigram(self, w, w_prev):
        if w_prev not in self.bigram_data:
            self.bigram_data[w_prev] = {}
            self.bigram_data_size[w_prev] = 0
        self.bigram_data_size[w_prev] += 1
        w_prev_dict = self.bigram_data[w_prev]
        if w in w_prev_dict:
            w_prev_dict[w] += 1
        else:
            w_prev_dict[w] = 1

    def load_data(self):
        self.nlp = spacy.load("en_core_web_sm")
        dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split="train")
        for text in dataset['text']:
            self.m +=1
            doc = self.nlp(text)
            w_prev = 'START'
            for w in doc:
                if w.is_alpha:
                    self.M +=1
                    self.add_to_unigram(w.lemma_)
                    self.add_to_bigram(w.lemma_, w_prev)
                    w_prev = w.lemma_  # todo: check first word in extreme cases is indeed START



In [39]:
class NGramModel:

    def __init__(self, corpus, n):
        self.n = n
        self.M = corpus.M #total num of tokens
        self.m = corpus.m #total num of sentences
        if n == 1:
            self.corpus = corpus.unigram_data  # {w: count / N}
            print(self.corpus)
            self.max_prob = - np.inf
            self.max_prob_word = None
            self.total_count = corpus.unigram_data_size
        elif n == 2:
            self.corpus = corpus.bigram_data
            self.max_probs = {} #{last_word : [MAX_PROB , NEXT_WORD]}
            self.total_counts = corpus.bigram_data_size # {w_{i-1} : count of pairs (w_{i-1} , w_j)}

    def train_unigram(self):
        for w_i in self.corpus.keys():
            count = self.corpus[w_i]
            prob_w_i = np.log(count / self.total_count)
            self.corpus[w_i] = prob_w_i
            if prob_w_i > self.max_prob:
                self.max_prob = prob_w_i
                self.max_prob_word = w_i

    def train_bigram(self):
        for w_i  in self.corpus.keys():
            self.max_probs[w_i] = [- np.inf, None]
            for w_j in self.corpus[w_i].keys():
                count = self.corpus[w_i][w_j]
                prob_w_j = np.log(count / self.total_counts[w_i])
                self.corpus[w_i][w_j] = prob_w_j
                if prob_w_j > self.max_probs[w_i][PROB]:
                    self.max_probs[w_i][PROB] = prob_w_j
                    self.max_probs[w_i][WORD] = w_j
        print(self.corpus['the'])

    def train(self):
        if self.n == 1:
            self.train_unigram()
        elif self.n == 2:
            self.train_bigram()

    def _predict_unigram(self, sentence):
        return self.max_prob_word

    def _predict_bigram(self, sentence):
        last_word = sentence[-1]
        print(self.corpus['in'])
        if last_word not in self.corpus:
            return "STOP"
        return self.max_probs[last_word][WORD]

    def predict(self, sentence):
        if self.n == 1:
            return self._predict_unigram(sentence)
        elif self.n == 2:
            return self._predict_bigram(sentence)

    def _probability_unigram(self, sentence):
        prob = 0
        for w in sentence:
            if w not in self.corpus:
                return 0
            prob += self.corpus[w]
        return prob

    def _probability_bigram(self, sentence):
        if sentence[0] not in self.corpus['START']:
            return 0
        prob = self.corpus['START'][sentence[0]]
        for i in range(2, len(sentence)):
            w_prev = sentence[i - 1]
            w_i = sentence[i]
            if w_prev not in self.corpus:
                return 0
            if w_i not in self.corpus[w_prev]:
                return 0
            prob += self.corpus[w_prev][w_i]
        return prob

    def probability(self, sentence):
        if self.n == 1:
            return self._probability_unigram(sentence)
        elif self.n == 2:
            return self._probability_bigram(sentence)

    def perplexity(self):
        pass

In [40]:
class LinearInterpolation:
    def __init__(self, unigram, bigram):
        self.unigram_model = unigram
        self.unigram_model = bigram

    def probability(self, sentence):
        pass

    def perplexity(self):
        pass

In [41]:
corpus = Corpus()
corpus.load_data()

Using the latest cached version of the module from /Users/mayagoldman/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Fri Nov 11 17:05:36 2022) since it couldn't be found locally at wikitext., or remotely on the Hugging Face Hub.
Found cached dataset wikitext (/Users/mayagoldman/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


In [42]:
print(corpus.unigram_data)

{'Valkyria': 54, 'Chronicles': 43, 'III': 229, 'Senjō': 5, 'no': 1648, 'Unrecorded': 1, 'chronicle': 32, 'Japanese': 48, 'light': 422, 'of': 57032, 'the': 130760, 'Battlefield': 4, 'commonly': 119, 'refer': 375, 'to': 39551, 'as': 15058, 'outside': 307, 'Japan': 212, 'be': 54359, 'a': 36064, 'tactical': 27, 'role': 737, 'play': 1890, 'video': 707, 'game': 2524, 'develop': 631, 'by': 12718, 'Sega': 9, 'and': 50737, 'Media': 40, 'Vision': 13, 'for': 13795, 'PlayStation': 125, 'Portable': 13, 'release': 1582, 'in': 45022, 'January': 693, 'it': 9271, 'third': 679, 'series': 1330, 'employ': 169, 'same': 979, 'fusion': 47, 'real': 237, 'time': 2978, 'gameplay': 101, 'its': 3878, 'predecessor': 79, 'story': 686, 'run': 1069, 'parallel': 93, 'first': 4017, 'follow': 1670, 'nameless': 5, 'penal': 5, 'military': 510, 'unit': 433, 'serve': 634, 'nation': 165, 'Gallia': 5, 'during': 2690, 'Second': 145, 'Europan': 3, 'War': 764, 'who': 3023, 'perform': 642, 'secret': 97, 'black': 412, 'operation':

In [43]:
"""1. Train maximum-likelihood unigram and bigram language models based on the above training data."""

unigram_model = NGramModel(corpus, UNIGRAM)


{'Valkyria': 54, 'Chronicles': 43, 'III': 229, 'Senjō': 5, 'no': 1648, 'Unrecorded': 1, 'chronicle': 32, 'Japanese': 48, 'light': 422, 'of': 57032, 'the': 130760, 'Battlefield': 4, 'commonly': 119, 'refer': 375, 'to': 39551, 'as': 15058, 'outside': 307, 'Japan': 212, 'be': 54359, 'a': 36064, 'tactical': 27, 'role': 737, 'play': 1890, 'video': 707, 'game': 2524, 'develop': 631, 'by': 12718, 'Sega': 9, 'and': 50737, 'Media': 40, 'Vision': 13, 'for': 13795, 'PlayStation': 125, 'Portable': 13, 'release': 1582, 'in': 45022, 'January': 693, 'it': 9271, 'third': 679, 'series': 1330, 'employ': 169, 'same': 979, 'fusion': 47, 'real': 237, 'time': 2978, 'gameplay': 101, 'its': 3878, 'predecessor': 79, 'story': 686, 'run': 1069, 'parallel': 93, 'first': 4017, 'follow': 1670, 'nameless': 5, 'penal': 5, 'military': 510, 'unit': 433, 'serve': 634, 'nation': 165, 'Gallia': 5, 'during': 2690, 'Second': 145, 'Europan': 3, 'War': 764, 'who': 3023, 'perform': 642, 'secret': 97, 'black': 412, 'operation':

In [49]:
bigram_model = NGramModel(corpus, BIGRAM)

In [45]:
unigram_model.train_unigram()

In [50]:
bigram_model.train_bigram()

/var/folders/0v/kptkrsdn7f3crt7djpgqm5cr0000gn/T/ipykernel_34272/2597464474.py:32: RuntimeWarning: invalid value encountered in log
  prob_w_j = np.log(count / self.total_counts[w_i])
/var/folders/0v/kptkrsdn7f3crt7djpgqm5cr0000gn/T/ipykernel_34272/2597464474.py:32: RuntimeWarning: divide by zero encountered in log
  prob_w_j = np.log(count / self.total_counts[w_i])


{'Battlefield': nan, 'PlayStation': nan, 'third': nan, 'Valkyria': nan, 'same': nan, 'story': nan, 'first': nan, 'nameless': nan, 'nation': nan, 'Second': nan, 'imperial': nan, 'game': nan, 'work': nan, 'standard': nan, 'series': nan, 'script': nan, 'franchise': nan, 'development': nan, 'player': nan, 'route': nan, 'map': nan, 'other': nan, 'main': nan, 'rest': nan, 'BliTZ': nan, 'battlefield': nan, 'character': nan, 'expense': nan, 'stat': nan, 'squad': nan, 'entire': nan, 'Nameless': nan, 'record': nan, 'gallian': nan, 'most': nan, 'Regular': nan, 'task': nan, 'three': nan, 'upper': nan, 'Gallian': nan, 'concept': nan, 'war': nan, 'great': nan, 'rank': nan, 'ideal': nan, 'presence': nan, 'desperately': nan, 'capital': nan, 'weary': nan, 'real': nan, 'major': nan, 'end': nan, 'Empire': nan, 'support': nan, 'chance': nan, 'empire': nan, 'two': nan, 'confront': nan, 'valkyrian': nan, 'director': nan, 'release': nan, 'staff': nan, 'popular': nan, 'team': nan, 'mechanic': nan, 'revolution

In [51]:
"""2. Using the bigram model, continue the following sentence with the most probable word predicted by the model: “ I
have a house in ... """
sentence = corpus.nlp('I have a house in')
predicted_word = bigram_model.predict(sentence)
print(predicted_word)

{'January': nan, 'Japan': nan, 'the': nan, 'carry': nan, 'November': nan, 'medium': nan, 'mission': nan, 'a': nan, 'third': nan, 'battle': nan, 'order': nan, 'manpower': nan, 'create': nan, 'action': nan, 'early': nan, 'an': nan, 'addition': nan, 'development': nan, 'particular': nan, 'September': nan, 'its': nan, 'flux': nan, 'February': nan, 'second': nan, 'story': nan, 'Moscow': nan, 'two': nan, 'Dengeki': nan, 'MacArthur': nan, 'downtown': nan, 'it': nan, 'central': nan, 'response': nan, 'and': nan, 'his': nan, 'this': nan, 'Little': nan, 'connection': nan, 'any': nan, 'person': nan, 'as': nan, 'time': nan, 'June': nan, 'at': nan, 'April': nan, 'May': nan, 'small': nan, 'aid': nan, 'Arkansas': nan, 'such': nan, 'August': nan, 'hope': nan, 'charge': nan, 'Vol': nan, 'service': nan, 'use': nan, 'obedience': nan, 'after': nan, 'Douglas': nan, 'favor': nan, 'North': nan, 'Big': nan, 'Works': nan, 'due': nan, 'in': nan, 'honor': nan, 'Union': nan, 'girlhood': nan, 'similar': nan, 'colla

In [48]:
""" 3. Using the bigram model:
(a) compute the probability of the following two sentences (for each sentence separately).
(b) compute the perplexity of both the following two sentences (treating them as a single test set with 2 sentences).

Brad Pitt was born in Oklahoma
The actor was born in USA
"""
sentence1 = corpus.nlp('Brad Pitt was born in Oklahoma')
sentence2 = corpus.nlp('The actor was born in USA')

sentence1_prob = unigram_model.probability(sentence1)
sentence2_prob = unigram_model.probability(sentence2)
print(f'sentence 1 probability:{sentence1_prob}')
print(f'sentence 2 probability:{sentence2_prob}')

sentence 1 probability:0
sentence 2 probability:0
